In [25]:
import pandas

df = pandas.DataFrame({"score_1": [0.1, 0.4, 0.42, 0.12, 0.94, 0.32, 0.5], "score_2": [0.52, 0.32, 0.01, 0.99, 0.56, 0.15, 0.69]})

print("Initial DataFrame")
print(df)

def mark_highlighted(row):
    if row.score_1 < 0.35 and row.score_2 < 0.35:
        return True
    if row.score_1 < 0.20 and row.score_2 < 0.90:
        return True
    if row.score_1 < 0.15 and row.score_2 < 0.80:
        return True

    return False
    

df["highlighted"] = df.aggregate(axis="columns", func=mark_highlighted)

print()
print("Adding highlighted to DataFrame")
print(df)

def assign_category(val):
    if val < 0.1:
        return "Very Low"
    if val >= 0.1 and val < 0.3:
        return "Medium"
    if val >= 0.3 and val < 0.8:
        return "High"
    return "Very High"

cat_list = []
for row in df["score_1"]:
    cat_list.append(assign_category(row))

cat = pandas.Categorical(cat_list, categories=["Very Low", "Medium", "High", "Very High"])
df["risk_1_group"] = cat

print()
print("Added risk_1_group categorical column")
print(df)

Initial DataFrame
   score_1  score_2
0     0.10     0.52
1     0.40     0.32
2     0.42     0.01
3     0.12     0.99
4     0.94     0.56
5     0.32     0.15
6     0.50     0.69

Adding highlighted to DataFrame
   score_1  score_2  highlighted
0     0.10     0.52         True
1     0.40     0.32        False
2     0.42     0.01        False
3     0.12     0.99        False
4     0.94     0.56        False
5     0.32     0.15         True
6     0.50     0.69        False

Added risk_1_group categorical column
   score_1  score_2  highlighted risk_1_group
0     0.10     0.52         True       Medium
1     0.40     0.32        False         High
2     0.42     0.01        False         High
3     0.12     0.99        False       Medium
4     0.94     0.56        False    Very High
5     0.32     0.15         True         High
6     0.50     0.69        False         High


In [68]:
from random import randint
import unittest


class Ad:
    type = "0000"
    cost_share = 0.0

    def __init__(self, spent=0):
        self.spent = spent

    @property
    def reimbursement(self):
        return round(self.spent * self.cost_share, 2)
    
    def __str__(self):
        cs = "{0:.2f}".format(self.cost_share)
        sp = "{0:.2f}".format(self.spent)
        rb = "{0:.2f}".format(self.reimbursement)
        return f"{self.type}\t{cs}\t\t{sp}\t\t{rb}"


class AdType0011(Ad):
    type = "0011"
    cost_share = 0.5


class AdType1011(Ad):
    type = "1011"
    cost_share = 1.0


class AdType1111(Ad):
    type = "1111"
    cost_share = 0.75


class AdType1010(Ad):
    type = "1010"
    cost_share = 0.9


class Reimbursement:
    def __init__(self):
        self._ads = []

    def add_ad(self, ad):
        self._ads.append(ad)

    def remove_ad(self, ad):
        self._ads.remove(ad)
    
    def get_ads(self, ad_type = None):
        if ad_type is None:
            return self._ads
        else:
            return list(filter(lambda ad: ad.type == ad_type, self._ads))
    
    def count_ads(self, ad_type = None):
        return len(self.get_ads(ad_type))

    def print_ads(self, ad_type = None):
        print("Ad_Type\tCost_Share_Rate\tActual_Spend\tReimbursement")
        ads = self.get_ads(ad_type)
        for ad in ads:
            print(ad)

    def get_total_reimbursement(self, ad_type = None):
        reimbursement = 0
        ads = self.get_ads(ad_type)
        for ad in ads:
            reimbursement += ad.reimbursement

        return round(reimbursement, 2)


class TestAds(unittest.TestCase):
    def test_type(self):
        ad0011 = AdType0011(200)
        ad1011 = AdType1011(1200)
        ad1111 = AdType1111(500)
        ad1010 = AdType1010(489)

        self.assertEqual(ad0011.type, "0011")
        self.assertEqual(ad1011.type, "1011")
        self.assertEqual(ad1111.type, "1111")
        self.assertEqual(ad1010.type, "1010")

    def test_cost_share(self):
        ad0011 = AdType0011(200)
        ad1011 = AdType1011(1200)
        ad1111 = AdType1111(500)
        ad1010 = AdType1010(489)

        self.assertEqual(ad0011.cost_share, 0.5)
        self.assertEqual(ad1011.cost_share, 1.0)
        self.assertEqual(ad1111.cost_share, 0.75)
        self.assertEqual(ad1010.cost_share, 0.9)
    
    def test_reimbursement(self):
        ad0011 = AdType0011(200)
        ad1011 = AdType1011(1200)
        ad1111 = AdType1111(500)
        ad1010 = AdType1010(489)

        self.assertEqual(ad0011.reimbursement, 200 * 0.5)
        self.assertEqual(ad1011.reimbursement, 1200 * 1.0)
        self.assertEqual(ad1111.reimbursement, 500 * 0.75)
        self.assertEqual(ad1010.reimbursement, 489 * 0.9)


class TestReimbursement(unittest.TestCase):
    def test_init(self):
        r = Reimbursement()

        self.assertEqual(r.count_ads(), 0)
        self.assertEqual(r.get_total_reimbursement(), 0)
        self.assertEqual(len(r.get_ads()), 0)

    def test_add_ad(self):
        r = Reimbursement()

        ad = AdType1111(500)
        r.add_ad(ad)

        self.assertEqual(r.count_ads(), 1)
        self.assertIs(r.get_ads()[0], ad)
    
    def test_removing_ad(self):
        r = Reimbursement()

        ad = AdType1111(500)
        ad2 = AdType1011(1500)

        r.add_ad(ad)
        r.add_ad(ad2)

        self.assertEqual(r.count_ads(), 2)
        r.remove_ad(ad2)
        self.assertEqual(r.count_ads(), 1)

    def test_get_ads(self):
        r = Reimbursement()

        ad = AdType1111(500)
        ad2 = AdType1011(1500)
        ad3 = AdType1011(1300)

        r.add_ad(ad)
        r.add_ad(ad2)
        r.add_ad(ad3)

        self.assertIs(r.get_ads()[0], ad)
        self.assertIs(r.get_ads()[1], ad2)
        self.assertIs(r.get_ads("1011")[0], ad2)

    def test_count_ads(self):
        r = Reimbursement()

        r.add_ad(AdType1111(500))
        r.add_ad(AdType1011(1500))
        r.add_ad(AdType1011(1200))

        self.assertEqual(r.count_ads(), 3)
        self.assertEqual(r.count_ads("1011"), 2)

    def test_get_total_reimbursement(self):
        r = Reimbursement()

        r.add_ad(AdType0011(200))
        r.add_ad(AdType1111(500))
        r.add_ad(AdType1011(1500))
        r.add_ad(AdType1011(1200))

        total_reimbursement = 200 * 0.5 + 500 * 0.75 + 1500 * 1 + 1200 * 1
        
        self.assertEqual(r.get_total_reimbursement(), total_reimbursement)
        self.assertEqual(r.get_total_reimbursement("1011"), 1500 + 1200) 


unittest.main(argv=[''], verbosity=2, exit=False)


test_cost_share (__main__.TestAds.test_cost_share) ... ok
test_reimbursement (__main__.TestAds.test_reimbursement) ... ok
test_type (__main__.TestAds.test_type) ... ok
test_add_ad (__main__.TestReimbursement.test_add_ad) ... ok
test_count_ads (__main__.TestReimbursement.test_count_ads) ... ok
test_get_ads (__main__.TestReimbursement.test_get_ads) ... ok
test_get_total_reimbursement (__main__.TestReimbursement.test_get_total_reimbursement) ... ok
test_init (__main__.TestReimbursement.test_init) ... ok
test_removing_ad (__main__.TestReimbursement.test_removing_ad) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.014s

OK


In [71]:
import pandas

csv = pandas.read_csv('task3_dateset.csv')

print(type(csv))

<class 'pandas.core.frame.DataFrame'>
